# Simple ResponseDataNode Query

This notebook demonstrates a straightforward programmatic query to retrieve the most recent ResponseDataNode instances from the AiiDA database.

**Purpose**: Query and display the 10 most recent ResponseDataNode instances with their details.

## 1. Load AiiDA Profile

Ensure the AiiDA profile is loaded:

In [1]:
import aiida
from aiida.manage import get_manager

# Load the presto-1 profile
try:
    # Check if a profile is already loaded
    manager = get_manager()
    current_profile = manager.get_profile()
    print(f"✓ AiiDA profile already loaded: {current_profile.name}")
    
    # Switch to presto-1 if needed
    if current_profile.name != 'presto-1':
        print(f"Switching to profile 'presto-1'...")
        aiida.load_profile('presto-1')
        print("✓ Profile switched successfully")
        
except Exception:
    # No profile loaded, load presto-1
    print("Loading AiiDA profile 'presto-1'...")
    aiida.load_profile('presto-1')
    print("✓ Profile loaded successfully")

# Verify profile is working
from aiida.orm import QueryBuilder, Node
qb = QueryBuilder()
qb.append(Node)
total_nodes = qb.count()
print(f"\nDatabase contains {total_nodes} total nodes")

Loading AiiDA profile 'presto-1'...
✓ Profile loaded successfully
✓ Profile loaded successfully

Database contains 814 total nodes

Database contains 814 total nodes


## 2. Query Recent ResponseDataNode Instances

Retrieve the 10 most recent ResponseDataNode instances from the database:

In [2]:
# Query for the 10 most recent ResponseDataNode instances
from aiida.orm import QueryBuilder, load_node
import pandas as pd
from IPython.display import display, HTML

print("=== Querying Recent ResponseDataNode Instances ===")

try:
    # Import ResponseDataNode
    from bmcs_matmod.gsm_lagrange.aiida_plugin.response_data_node import ResponseDataNode
    
    # Create query for the 10 most recent instances
    qb = QueryBuilder()
    qb.append(ResponseDataNode, project=['pk', 'uuid', 'ctime'])
    qb.order_by({ResponseDataNode: {'ctime': 'desc'}})  # Most recent first
    qb.limit(10)  # Limit to 10 results
    
    # Execute query
    results = qb.all()
    
    print(f"✅ Found {len(results)} ResponseDataNode instances")
    
    if results:
        # Create DataFrame for nice display
        data = []
        for pk, uuid, ctime in results:
            # Load the actual node to access more details
            node = load_node(pk)
            
            # Try to get array names if available
            try:
                if hasattr(node, 'get_arraynames'):
                    arrays = ', '.join(node.get_arraynames())
                else:
                    arrays = 'N/A'
            except:
                arrays = 'Error'
            
            data.append({
                'Node PK': pk,
                'UUID (short)': str(uuid)[:8] + '...',
                'Created': ctime.strftime('%Y-%m-%d %H:%M:%S'),
                'Type': node.__class__.__name__,
                'Arrays': arrays
            })
        
        # Create DataFrame and display as HTML table
        df = pd.DataFrame(data)
        
        print("\n📊 Recent ResponseDataNode Instances:")
        display(HTML(df.to_html(index=False)))
        
        # Store results for further analysis
        recent_nodes = [load_node(pk) for pk, _, _ in results]
        print(f"\n✅ Results stored in 'recent_nodes' variable for further analysis")
        
    else:
        print("❌ No ResponseDataNode instances found in the database")
        recent_nodes = []

except ImportError as e:
    print(f"❌ ResponseDataNode import failed: {e}")
    print("Make sure the ResponseDataNode module is available")
    recent_nodes = []
except Exception as e:
    print(f"❌ Query failed: {e}")
    recent_nodes = []

=== Querying Recent ResponseDataNode Instances ===
✅ Found 10 ResponseDataNode instances
✅ Found 10 ResponseDataNode instances

📊 Recent ResponseDataNode Instances:

📊 Recent ResponseDataNode Instances:


Node PK,UUID (short),Created,Type,Arrays
814,1c730e66...,2025-08-18 18:20:12,ResponseDataNode,"time, eps, sig, iterations, lambda, Eps_t_flat, Sig_t_flat, Eps_omega_a, Eps_z_a, Sig_Y_a, Sig_Z_a"
813,df4fe7df...,2025-08-18 17:42:13,ResponseDataNode,"time, eps, sig, iterations, lambda, Eps_t_flat, Sig_t_flat, Eps_omega_a, Eps_z_a, Sig_Y_a, Sig_Z_a"
812,63bbceff...,2025-08-18 17:37:24,ResponseDataNode,"time, eps, sig, iterations, lambda, Eps_t_flat, Sig_t_flat, Eps_omega_a, Eps_z_a, Sig_Y_a, Sig_Z_a"
811,6adf2f66...,2025-08-16 12:46:07,ResponseDataNode,"time, eps, sig, iterations, lambda, Eps_t_flat, Sig_t_flat, Eps_omega_a, Eps_z_a, Sig_Y_a, Sig_Z_a"
810,f4eeb599...,2025-07-01 12:17:33,ResponseDataNode,"time, eps, sig, iterations, lambda, Eps_t_flat, Sig_t_flat, Eps_omega_a, Eps_z_a, Sig_Y_a, Sig_Z_a"
809,4f7a9247...,2025-07-01 12:07:36,ResponseDataNode,"time, eps, sig, iterations, lambda, Eps_t_flat, Sig_t_flat, Eps_omega_a, Eps_z_a, Sig_Y_a, Sig_Z_a"
808,25cca357...,2025-07-01 12:06:30,ResponseDataNode,"time, eps, sig, iterations, lambda, Eps_t_flat, Sig_t_flat, Eps_omega_a, Eps_z_a, Sig_Y_a, Sig_Z_a"
807,b8668daa...,2025-07-01 11:54:51,ResponseDataNode,"time, eps, sig, iterations, lambda, Eps_t_flat, Sig_t_flat, Eps_omega_a, Eps_z_a, Sig_Y_a, Sig_Z_a"
806,0a59e5d0...,2025-07-01 11:53:15,ResponseDataNode,"time, eps, sig, iterations, lambda, Eps_t_flat, Sig_t_flat, Eps_omega_a, Eps_z_a, Sig_Y_a, Sig_Z_a"
805,3550cc4c...,2025-07-01 11:52:21,ResponseDataNode,"time, eps, sig, iterations, lambda, Eps_t_flat, Sig_t_flat, Eps_omega_a, Eps_z_a, Sig_Y_a, Sig_Z_a"



✅ Results stored in 'recent_nodes' variable for further analysis


## 3. Detailed Analysis of Results

Examine the first result in detail:

In [ ]:
# Detailed analysis of the first (most recent) result
if recent_nodes:
    print("=== Detailed Analysis of Most Recent ResponseDataNode ===")
    
    node = recent_nodes[0]
    print(f"\n🔍 Node Details:")
    print(f"  PK: {node.pk}")
    print(f"  UUID: {node.uuid}")
    print(f"  Created: {node.ctime}")
    print(f"  Type: {node.__class__.__name__}")
    
    # Try to access array data
    if hasattr(node, 'get_arraynames'):
        arrays = node.get_arraynames()
        print(f"\n📊 Available Arrays: {arrays}")
        
        # Show array details
        for array_name in arrays:
            try:
                array = node.get_array(array_name)
                print(f"  {array_name}: shape {array.shape}, dtype {array.dtype}")
                
                # Show sample values for small arrays
                if array.size <= 10:
                    print(f"    Values: {array.flatten()}")
                else:
                    print(f"    Range: {array.min():.3f} to {array.max():.3f}")
                    
            except Exception as e:
                print(f"  {array_name}: Error accessing array - {e}")
    
    # Show metadata using AiiDA v3 syntax
    attributes = node.base.attributes.all
    if attributes:
        print(f"\n📝 Attributes: {list(attributes.keys())}")
        
        # Show simulation metadata if available
        if 'simulation_metadata' in attributes:
            metadata = attributes['simulation_metadata']
            print(f"  Simulation metadata keys: {list(metadata.keys())}")
            
else:
    print("❌ No nodes available for detailed analysis")

=== Detailed Analysis of Most Recent ResponseDataNode ===

🔍 Node Details:
  PK: 814
  UUID: 1c730e66-87b5-44f4-9668-a307e430d7b3
  Created: 2025-08-18 18:20:12.084923+00:00
  Type: ResponseDataNode

📊 Available Arrays: ['time', 'eps', 'sig', 'iterations', 'lambda', 'Eps_t_flat', 'Sig_t_flat', 'Eps_omega_a', 'Eps_z_a', 'Sig_Y_a', 'Sig_Z_a']
  time: shape (601,), dtype float64
    Range: 0.000 to 3.000
  eps: shape (601, 1), dtype float64
    Range: 0.000 to 0.001
  sig: shape (601, 1, 1), dtype float64
    Range: 0.000 to 23.386
  iterations: shape (601, 1), dtype int64
    Range: 0.000 to 1.000
  lambda: shape (601, 1, 1), dtype float64
    Range: 0.000 to 0.002
  sig: shape (601, 1, 1), dtype float64
    Range: 0.000 to 23.386
  iterations: shape (601, 1), dtype int64
    Range: 0.000 to 1.000
  lambda: shape (601, 1, 1), dtype float64
    Range: 0.000 to 0.002
  Eps_t_flat: shape (601, 2), dtype float64
    Range: 0.000 to 0.026
  Sig_t_flat: shape (601, 2), dtype float64
    Range:

## 4. Summary Statistics

Show summary information about all retrieved nodes:

In [4]:
# Summary statistics for all retrieved nodes
if recent_nodes:
    print("=== Summary Statistics ===")
    print(f"\nTotal nodes retrieved: {len(recent_nodes)}")
    
    # Creation time analysis
    creation_times = [node.ctime for node in recent_nodes]
    earliest = min(creation_times)
    latest = max(creation_times)
    
    print(f"\n📅 Time Range:")
    print(f"  Earliest: {earliest.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"  Latest: {latest.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"  Span: {latest - earliest}")
    
    # Array analysis
    print(f"\n📊 Array Analysis:")
    total_arrays = 0
    array_names_set = set()
    
    for node in recent_nodes:
        if hasattr(node, 'get_arraynames'):
            try:
                arrays = node.get_arraynames()
                total_arrays += len(arrays)
                array_names_set.update(arrays)
            except:
                pass
    
    print(f"  Total arrays across all nodes: {total_arrays}")
    print(f"  Unique array names: {sorted(array_names_set)}")
    print(f"  Average arrays per node: {total_arrays / len(recent_nodes):.1f}")
    
    print(f"\n✅ Query completed successfully!")
    print(f"Use 'recent_nodes' variable to access the loaded ResponseDataNode instances.")
    
else:
    print("❌ No summary available - no nodes were retrieved")

=== Summary Statistics ===

Total nodes retrieved: 10

📅 Time Range:
  Earliest: 2025-07-01 11:52:21
  Latest: 2025-08-18 18:20:12
  Span: 48 days, 6:27:50.417244

📊 Array Analysis:
  Total arrays across all nodes: 110
  Unique array names: ['Eps_omega_a', 'Eps_t_flat', 'Eps_z_a', 'Sig_Y_a', 'Sig_Z_a', 'Sig_t_flat', 'eps', 'iterations', 'lambda', 'sig', 'time']
  Average arrays per node: 11.0

✅ Query completed successfully!
Use 'recent_nodes' variable to access the loaded ResponseDataNode instances.


## Usage Notes

This notebook provides a simple, direct approach to querying ResponseDataNode instances from the AiiDA database.

**Key Features:**
- Retrieves the 10 most recent ResponseDataNode instances
- Displays results in a formatted table
- Provides detailed analysis of the most recent node
- Shows summary statistics across all retrieved nodes

**Variables Available:**
- `recent_nodes`: List of the 10 most recent ResponseDataNode instances
- `df`: Pandas DataFrame with summary information

**Next Steps:**
- Use the `recent_nodes` list for further analysis
- Access individual arrays with `node.get_array('array_name')`
- Convert back to ResponseData objects if needed with `node.to_response_data()` (if implemented)